# Adversarial Examples - LeNet on MNIST Data

LeNet style network on the development set of MNIST dataset. 10,000 images.

In [2]:
## Imports
import numpy as np
import pandas as pd

from PIL import Image
import torch
import torch.nn as nn
from torchvision import models
from torchvision import datasets
import random

import torch.nn.functional as F

import matplotlib.pyplot as plt

from torchvision import transforms
import urllib

plt.rcParams['figure.figsize'] = (20.0, 10.0)

from IPython.display import clear_output

## 1. Load data and model

First we load the model and data.


### 1.1 Model

LeNet

In [3]:
## Load model and data
pretrained_model = "models/lenet_mnist_model.pth"

## Define LeNet model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)
    
## Initialize the network
model = Net()

## Load the pretrained model
model.load_state_dict(torch.load(pretrained_model, map_location='cpu'))

## Set the model in evaluation mode. In this case this is for the Dropout layers
model.eval()

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

### 1.2 Data

Loading the data into a PyTorch dataloader. Here we also integrate the scaling transformation step.

We also define the function `show_tensor_image`to display an image.

In [4]:
data_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, download=True, transform=transforms.Compose([
            transforms.ToTensor(),
            ])), 
        batch_size=1, shuffle=True)

In [5]:
data_loader.dataset.targets[2]

tensor(1)

In [6]:
def image_to_tensor(data_loader, image_number="random"):
    '''
    Returns a random image of given label as tensor object
    
    Inputs:
    data_loader  -- PyTorch dataloader with the MNIST data
    image_number -- Number of example to show. There are 10,000. Default is random
    
    Returns:
    image        -- Image as tensor of dimension (1, 1, 28, 28)
    label        -- Label as integer
    '''
    
    if image_number == "random":
        i = random.randrange(0, 10000)
    else:
        i = int(image_number)
    
    ## Extract data
    image = data_loader.dataset.data[i,:,:]

    ## Add batch and channel dimension as required by net
    image = image.unsqueeze(0).unsqueeze(0)
    image = image.float()
    
    ## Extract label
    label = int(data_loader.dataset.targets[i].numpy())
       
    ## Print image number
    print("Image number {} chosen\n".format(int(i)))
    
    
    return image, label

In [7]:
def show_tensor_image(tensor):
    '''
    Plots the given image as tensor.
    
    Inputs:
    tensor -- image as tensor of dimension (1, 28, 28)
    
    Returns:
    Plot of the image
    '''
    
    ## Remove batch and channel dimension
    tensor = tensor.detach()
    tensor = tensor.squeeze(0).squeeze(0)

    ## Plot
    plt.imshow(tensor.numpy(), cmap="gray")

## 2. Predict

In [ ]:
def predict(model, image, label, return_grad=False, supress_output=True):
    '''
    Predicts
    
    Inputs:
    model          -- net
    image          -- Input image as tensor of shape (1, 1, 28, 28)
    label          -- class label as tensor
    return_grad    -- Returns gradient if set True
    supress_output -- Prints output if set False
    
    Returns:
    gradient       -- None if return_grad=False. Otherwise the gradient from the prediction as a tensor. 
    accuracy       -- Integer of value 1 if class is panda, otherwise 0
    confidence     -- Confidence of prediction
    '''
        
    if return_grad == True:
        image.requires_grad=True
        prediction = model(image)
               
        # Zero gradients
        model.zero_grad()

        # Calculate loss using the class index for pandas and get gradient
        loss = F.nll_loss(prediction, torch.tensor([label]))
        loss.backward()
        gradient = image.grad.data
        
    else:           
        gradient = None
        with torch.no_grad():
            prediction = model(image)
    
    ## Calculate if prediction is correct
    class_index = np.argmax(prediction.detach().numpy())
    
    if class_index == label:
        accuracy = 1.0
        
    else:
        accuracy = 0.0
     
    
    ## Get class index and confidence
    prediction = torch.nn.functional.softmax(prediction[0].detach(), dim=0).numpy()    
    confidence = prediction[class_index] * 100
    
    ## Get class name from the predicted index
    name = np.argmax(prediction)
   
    if supress_output == False:
        print("Predicted class:\n{}\n{:.2f} % confidence\n".format(name, confidence))
        print("Actual class: \n{}".format(label))
    
    return gradient, accuracy, confidence/100

## 3. Generate Adversarial Examples

Functions for generating the examples. The details are discussed in the next section.

In [ ]:
def generate_adversarial_fgsm(image, epsilon, grad_x):
    '''
    Applies FGSM attack on input image.
    
    Inputs:
    image       -- Image data as tensor
    epsilon     -- Hyperparameter
    grad_x      -- Gradient of the cost with respect to x
    
    Returns:
    image_tilde -- Adversarial image as tensor
    '''
    
    ## Compute eta part
    eta = epsilon * grad_x.sign()
    
    ## Apply perturbation
    image_tilde = image + eta
    
    ## Clip image to maintain the range [0, 1]
    image_tilde = torch.clamp(image_tilde, 0, 1)
    
    return image_tilde

## 4. Run Attacks

## 4.1 Fast Gradient Sign Method

This method by [1] generates adversarial examples quickly. It computes a step of gradient descent and moves one step of magnitude $\epsilon$ into the direction of this gradient:

$\widetilde{x} = x + \eta$

$\eta = \epsilon \cdot sign(\nabla_{x} J(\Theta, x, y))$

This method computes examples quickly where $\epsilon$ is a hyperparameter. Let's inspect a few clean and perturbed examples for small and large values.

In [8]:
def plot_example(image_as_tensor, label, epsilon):
    '''
    Generates an adversarial image. Plots the clean and adversarial image side-by-side
    
    Inputs:
    image_as_tensor -- Image as tensor
    label           -- Class label for target class
    epsilon         -- Hyperparameter for FGSM
    '''
    
    ## Predict for clean image
    gradient, _, _ = predict(model, image_as_tensor, label=label, return_grad=True, supress_output=False)

    print("\n==============================\nPerturbed image: \n")

    ## Compute adversarial image and predict for it.
    perturbed_data = generate_adversarial_fgsm(image_as_tensor, epsilon, gradient)
    predict(model, perturbed_data, label=label, return_grad=False, supress_output=False)


    ## Plots
    plt.subplot(121)
    plt.title("Clean example", fontsize=30)
    show_tensor_image(image_as_tensor)
    plt.subplot(122)
    plt.title("Adversarial example", fontsize=30)
    show_tensor_image(perturbed_data)

In [ ]:
## Random example at lower epsilon
epsilon = 12/255


image_as_tensor, label = image_to_tensor(data_loader)


plot_example(image_as_tensor, label, epsilon)

In [ ]:
## Random example at higher epsilon
epsilon = 50/255

image_as_tensor, label = image_to_tensor(data_loader, image_number=5761)
plot_example(image_as_tensor, label, epsilon)

As can be seen, for cetrain values of $\epsilon$ the perturbations are clearly visable to the human eye. At the same time, the exampels are still mostly classified as the correct class, however at a lower confidence.

For $\epsilon = 50/255$ sample number `5761` is an example for a desirable adversarial input. The predicted class is `2` with a confidence of over $96$%. However, the actual class is `9`.

Next we generate adversarial inputs with different values of $\epsilon$ and compute the average accuracy and confidence on all data.

In [ ]:
## Run on all 10000 examples
epsilons = [0, 1/255, 2/255, 4/255, 10/255, 20/255, 30/255, 40/255, 50/255, 60/255, 90/255]

accuracy = []
confidence = []

accuracy_adversarial = []
confidence_adversarial = []

for epsilon in epsilons:
   
    acc_sub = []
    conf_sub = []
    acc_sub_adver = []
    conf_sub_adver = []
    i = 1
    
    for image_as_tensor, label in data_loader:      
        clear_output(wait=True)
        print("Running for epsilon {:.2f}".format(epsilon))
        print(i)
        
        ## Predict with clean image
        gradient, acc, conf = predict(model, image_as_tensor, label=label, return_grad=True)
        acc_sub.append(acc)
        conf_sub.append(conf)

        ## Predict with adversarial image
        perturbed_data = generate_adversarial_fgsm(image_as_tensor, epsilon, gradient)
        _, acc, conf = predict(model, perturbed_data, label=label)
        acc_sub_adver.append(acc)
        conf_sub_adver.append(conf)
        i += 1
        
    ## Add accuracies and confidences
    accuracy.append(np.mean(acc_sub))
    confidence.append(np.mean(conf_sub))
    accuracy_adversarial.append(np.mean(acc_sub_adver))
    confidence_adversarial.append(np.mean(conf_sub_adver))


## Save results
result = pd.DataFrame()
e = np.array(epsilons) * 255
result["Epsilon_255"] = e
result["Accuracy"] = accuracy_adversarial
result["Confidence"] = confidence_adversarial
result.to_csv("results/LeNet_MNIST_FGSM.csv")

In [ ]:
plt.plot(e, accuracy_adversarial, "s-", color="navy", label="Accuracy")
plt.plot(e, confidence_adversarial, "^-", color="orange", label="Confidence")
plt.xlabel("Epsilon [*255]")

plt.title("Adversarial Examples on MNIST Dev Set", fontsize=30)
plt.legend()
#plt.show()

plt.savefig("Adversarial_Examples_MNIST_FGSM.pdf")
plt.savefig("Adversarial_Examples_MNIST_FGSM.png")

## 4.2 Basic Iterative Method (BIM)

In the previous section we found examples which are classified as the wrong class at a high confidence using the FGSM. To the human eye however, these examples can clearly be distinguished from the clean examples. Another problem with FGSM is that ist can be used for untargeted attacks only [2].

BIM requires the following function, which clips pixel values of an image $X^\prime$:

**TODO**
Fix explanation!!

Similar to the fast method we update the pixel values:

\begin{equation}
\tag{1}
\widetilde{X}_{n+1} = Clip_{X, \epsilon} \{ \widetilde{X}_{n} + \alpha sign(\nabla_{X} J(\widetilde{X}_{n}, Y_{true})) \}
\end{equation}




Here, the Clip function is defined as:

\begin{equation}
\tag{2}
Clip_{X, \epsilon} \{ X^\prime \} (x, y, z) = min\{ 255, X(x, y, z) + \epsilon, max\{0, X(x, y, z)-\epsilon, X^\prime(x, y, z) \} \}
\end{equation}

For the implementation we initialize:

\begin{equation}
\tag{3}
\widetilde{X}_{0} = X
\end{equation}

For the implementation we do for the number of iterations `n`:


a) Compute $ X^\prime = \widetilde{X}_{n} + \alpha sign(\nabla_{X} J(\widetilde{X}_{n}, Y_{true}))$ as used in equation (1)

b) Compute $ X(x, y, z) + \epsilon $ and $ X(x, y, z) - \epsilon $

c) Evaluate equation (2) using steps a) and b)

$max\{0, X(x, y, z)-\epsilon, X^\prime(x, y, z) \}$ (1.2)

d) Retrieve updated adversarial image $\widetilde{X}_{1}$ as given in equation (1)



In [52]:
## Test cases
epsilon = 10/255
alpha = 1/255
n = 5

X_test, Y_test = image_to_tensor(data_loader, image_number='random')

## Generate test image of shape (1, 1, 28, 28)
#X_ones = torch.ones((1, 1, 28, 28), dtype=torch.float)
X_adv = apply_BIM(model, X_test, Y_test, alpha, epsilon, num_iterations=n)

torch.equal(X_test, X_adv)

Image number 6695 chosen

0
tensor([[-1492.0259, -1172.3378, -1216.6344, -1427.5031,     0.0000, -1192.5981,
         -1109.0784, -1099.5205, -1029.8264,  -745.4501]],
       grad_fn=<LogSoftmaxBackward>)


AssertionError: 

In [51]:
def apply_BIM(model, image, label, alpha, epsilon, num_iterations=2):
    '''
    Applies given number of steps of the Basic Iterative Method (BIM) attack on the input image.
    
    Inputs:
    model          -- Model under attack
    image          -- Image data as tensor of shape (1, 1, 28, 28)
    label          -- Label from image of shape (1)
    alpha          -- Hyperparameter for iterative step??
    epsilon        -- Hyperparameter for sign method???
    num_iterations -- Number of iterations to perform
    
    Returns:
    image_adver    -- Adversarial image as tensor
    '''
    
    ## Convert label to torch tensor
    label = torch.tensor(label, dtype=torch.long).unsqueeze(0)    
    
    ## Check input image and label shapes
    assert(image.shape == torch.Size([1, 1, 28, 28]))
    assert(label.shape == torch.Size([1]))
    
    ## Initialize adversarial image as image according to equation (3)
    image_adver = image.clone()    
    #image_adver.requires_grad=True
    
    for i in range(num_iterations):
        print(i)
        
        ## Make a copy and detach so the computation graph can be constructed
        image_adver = image_adver.clone().detach()
        image_adver.requires_grad=True
        
        ## Compute cost with example image_adersarial        
        #image_adver.retain_grad=True
        
        pred = model(image_adver)
        
        print(pred)
        #class_index = np.argmax(pred.detach().numpy())
        
        loss = F.nll_loss(pred, label)        
        model.zero_grad()        
        loss.backward()        
        grad_x = image_adver.grad.data       
        
        ## Checks
        assert(image_adver.grad is not None)
        
        
        ## Compute X_prime according to equation (1)
        image_prime = image_adver + alpha * grad_x.detach().sign()
        assert(torch.equal(image_prime, image_adver) == False)
    
        ## Apply clip function to image_prime as in equation (2)       
        image_plus = image + epsilon
        image_minus = image - epsilon
        assert(torch.equal(image_plus, image) == False)
        
        # Equation 1.2
        third_part_1 = torch.max(image_minus, image_prime)
        third_part = torch.max(torch.tensor(0, dtype=torch.float), third_part_1)
              
        # Equation (2)
        image_adver = torch.min(image_plus, third_part)                 
        image_adver = torch.min(torch.tensor(255, dtype=torch.float), image_adver)                        

    
    return image_adver

## 5. Conclusions

Fast gradient sign method produces examples which can be detected by a human.

## References

[1] &emsp; Goodfellow et al. (2015) *Explaining and Harnessing Adversarial Examples*

[2] &emsp; Kurakin et al. (2017) *Adversarial examples in the physical world*